# Segmenting and Clustering Neighborhoods in Toronto

# Part 1

In [219]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from bs4 import BeautifulSoup
import requests
import urllib.request
from urllib.request import urlopen

In [220]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#### Scrape data into dataframe

In [221]:
html = urlopen(url)
soup = BeautifulSoup(html,'html.parser')
table1 = soup.find('table',cellpadding = '2')
PostalCode = table1.find_all('b')
df1 = pd.DataFrame([PostalCode])
df1 = df1.transpose()
span = table1.find_all('span')
df2 = pd.DataFrame(span,columns=['Borough'])

#### reformat dataframe to have 2 columns

In [222]:
df = pd.concat([df1,df2],axis = 1, sort = False)
df = df.rename(columns = {0:'PostalCode'})

#### Remove unassigned Postal codes

In [223]:
df['PostalCode'] = df['PostalCode'].astype(str)
df['Borough'] = df['Borough'].astype(str)
df = df[~df.Borough.str.contains("<i>Not assigned")]
df.head()

,PostalCode,Borough
2,<b>M3A</b>,"<span style=""font-size:80%;""><a href=""/wiki/No..."
3,<b>M4A</b>,"<span style=""font-size:80%;""><a href=""/wiki/No..."
4,<b>M5A</b>,"<span style=""font-size:80%;""><a href=""/wiki/Do..."
5,<b>M6A</b>,"<span style=""font-size:80%;""><a href=""/wiki/No..."
6,<b>M7A</b>,"<span style=""font-size:80%;""><a href=""/wiki/Qu..."


#### Seperate Borough and Neighborhood into 2 columns

In [224]:
df[['Borough','Neighborhood']] = df['Borough'].str.split('<br/>',n=1,expand = True)
df.head()

,PostalCode,Borough,Neighborhood
2,<b>M3A</b>,"<span style=""font-size:80%;""><a href=""/wiki/No...","(<a href=""/wiki/Parkwoods"" title=""Parkwoods"">P..."
3,<b>M4A</b>,"<span style=""font-size:80%;""><a href=""/wiki/No...","(<a href=""/wiki/Victoria_Village"" title=""Victo..."
4,<b>M5A</b>,"<span style=""font-size:80%;""><a href=""/wiki/Do...","(<a href=""/wiki/Regent_Park"" title=""Regent Par..."
5,<b>M6A</b>,"<span style=""font-size:80%;""><a href=""/wiki/No...","(<a href=""/wiki/Lawrence_Manor"" title=""Lawrenc..."
6,<b>M7A</b>,"<span style=""font-size:80%;""><a href=""/wiki/Qu...","/ <a class=""mw-redirect"" href=""/wiki/Ontario_..."


#### Remove html formatting, remove extra descriptors in 'Neighborhood', remove parenthesis and replace '/' with ','

In [225]:
df.replace(regex=True,inplace=True,to_replace='(<.*?>)',value=r'')
df = df.reset_index()
df.drop('index', axis = 1,inplace = True)
df['remove'] = df['Neighborhood'].replace(regex=True,to_replace=r'(\(.*?\))',value=r'')
df['Neighborhood'].replace(regex=True,inplace=True,to_replace=df['remove'],value=r'')
df.drop('remove', axis = 1, inplace = True)
df.replace(regex=True,inplace=True,to_replace=' /',value=r',')
df.replace(regex=True,inplace=True,to_replace='(\()',value=r'')
df.replace(regex=True,inplace=True,to_replace='(\))',value=r'')
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,


#### Assign empty neighborhoods to match borough

In [226]:
df['Neighborhood'].replace(regex=True,inplace=True,to_replace='',value=df['Borough'])
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park


In [227]:
df.shape

(103, 3)

# Part 2

#### Create dataframe with lat and long data from csv

In [228]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_6e10a103a1a0423a8955fa6fe0f98a16 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='FdFNrdnn-INE6-4YMDO2LQvcEB3osUh04HQhT4xB4nIu',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_6e10a103a1a0423a8955fa6fe0f98a16.get_object(Bucket='courseracapstone-donotdelete-pr-vvst8dr1yzrmk5',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Add lat and long data to existing df

In [229]:
df['Latitude'] = df_data_1['Latitude']
df['Longitude'] = df_data_1['Longitude']
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Queen's Park,Queen's Park,43.773136,-79.239476


# Part 3

In [230]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium

#### Make new dataframe containing data only of boroughs containing "Toronto"

In [231]:
dfTor = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
dfTor.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
3,M4E,East Toronto,The Beaches,43.786947,-79.385975
4,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714


#### Map Neighborhoods over Toronto

In [232]:
tlatitude = 43.7
tlongitude = -79.4
map_toronto = folium.Map(location=[tlatitude, tlongitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfTor['Latitude'], dfTor['Longitude'], dfTor['Borough'], dfTor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define FourSquare info

In [233]:
CLIENT_ID = 'Q3Q3QPKNRJENIJLVGSWTUWV3FXQVTG0OV0NU5TOI3MKVSSNB'
CLIENT_SECRET = 'VYRHGN2SL1EWH0A35UFU0QQLHEDRFWLAZQNTFVVLALQGC2AY'
VERSION = '20180605'

In [234]:
LIMIT = 100 

radius = 500 

In [235]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [236]:
toronto_venues = getNearbyVenues(names=dfTor['Neighborhood'],
                                   latitudes=dfTor['Latitude'],
                                   longitudes=dfTor['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North &amp; West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadian Place, Under

#### Analyze Neighborhoods by venues

In [237]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,...,Thrift / Vintage Store,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [238]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Thrift / Vintage Store,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.02439,0.0,0.0,0.0,0.0,0.0,0.04878,0.0,0.0,...,0.02439,0.0,0.0,0.0,0.0,0.0,0.0,0.02439,0.0,0.0
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.06250,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
3,Central Bay Street,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
4,Christie,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0


In [239]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                       venue  freq
0                  Cafeteria   1.0
1  Middle Eastern Restaurant   0.0
2  Latin American Restaurant   0.0
3         Light Rail Station   0.0
4             Lingerie Store   0.0


----Brockton, Parkdale Village, Exhibition Place----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.07
2              Brewery  0.05
3  American Restaurant  0.05
4               Bakery  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                  venue  freq
0    Light Rail Station  0.12
1                Garden  0.06
2  Fast Food Restaurant  0.06
3            Restaurant  0.06
4        Farmers Market  0.06


----Central Bay Street----
           venue  freq
0  Grocery Store  0.14
1    Coffee Shop  0.14
2       Pharmacy  0.14
3    Pizza Place  0.14
4        Butcher  0.14


----Christie----
                       venue  freq
0                      

In [240]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [241]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cafeteria,Women's Store,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Distribution Center
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bakery,Brewery,American Restaurant,Cheese Shop,Bank,Bar,Latin American Restaurant,Seafood Restaurant
2,"CN Tower, King and Spadina, Railway Lands, Har...",Light Rail Station,Yoga Studio,Auto Workshop,Park,Pizza Place,Burrito Place,Brewery,Restaurant,Farmers Market,Skate Park
3,Central Bay Street,Coffee Shop,Grocery Store,Discount Store,Bank,Butcher,Pharmacy,Pizza Place,Costume Shop,Coworking Space,Eastern European Restaurant
4,Christie,Park,Food & Drink Shop,Women's Store,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store


#### Cluster Neighborhoods

In [242]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 0, 0, 0, 0, 0, 4, 0, 0, 0], dtype=int32)

In [243]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dfTor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,0,Intersection,Rental Car Location,Spa,Mexican Restaurant,Bank,Medical Center,Electronics Store,Women's Store,Dessert Shop,Dumpling Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,0,College Stadium,General Entertainment,Skating Rink,Café,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Distribution Center
2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389,0,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Nail Salon,Breakfast Spot,Bank,Pizza Place,Pharmacy,Sandwich Place,Grocery Store
3,M4E,East Toronto,The Beaches,43.786947,-79.385975,0,Japanese Restaurant,Chinese Restaurant,Bank,Café,Women's Store,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714,1,Cafeteria,Women's Store,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Distribution Center


#### Visualize clusters

In [244]:
map_clusters = folium.Map(location=[tlatitude, tlongitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters